A.S. Lundervold, v101022

# Setup

In [1]:
%matplotlib inline

import numpy as np, pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path
import seaborn as sns 
import sklearn

In [2]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory
# or on Kaggle, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
try:
    import colab
    colab=True
except:
    colab=False

import os
kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [3]:
NB_DIR = Path.cwd()
DATA = NB_DIR/'data'
DATA.mkdir(exist_ok=True)

# Introduction

Machine learning models typically have several _hyperparameters_ we must choose correctly to achieve higher performance.

> **What is a hyperparameter?** During training, the model's _parameters_ are automatically tuned to make the model produce useful outputs. This is typically achieved by using an optimization algorithm (for example, gradient descent) to minimize some cost function (for example, mean square error or cross-entropy). 

> However, there are typically other parameters in the model that are not automatically tuned during training. These are the things you pass to the scikit-learn estimators as parameters. For example `RandomForestClassifier(max_depth = 2, n_estimators = 100, ...)` They are called _hyperparameters_. Examples include things like the learning rate, the amount of regularization, the number of layers in a neural network, and much more. Some models have a large number of such parameters, which can influence their performance heavily. 

**How do we select good hyperparameters?**

It's essentially a learning task: train the model to also obtain good hyperparameter settings. However, it's typically not that easy to formulate the task in a way where machine learning training methods can work (it's, for example, challenging to create cost functions for this task that can be optimized using gradient descent since such cost functions wouldn't be differentiable). 

> **There are some exciting methods to make powerful models that optimize ML models, but that is beyond our scope in this notebook**. Have a look at <a href="https://ai.googleblog.com/2017/05/using-machine-learning-to-explore.html">AutoML based on reinforcement learning</a> or <a href="https://en.wikipedia.org/wiki/Hyperparameter_optimization#Evolutionary_optimization">evolutionary algorithms</a> if you're curious.

**One approach to hyperparameter selection: simply search!**

# Searching for good hyperparameters

Two standard ways are 
1. **brute-force search**: try all parameter combinations within a specified range, and 
2. **random search**: try out random combinations of parameter settings within a specified range

A third way is to be more clever and use the results obtained from previous parameter settings to select the next setting that is expected to be better. This leads to things like 
3. **bayesian hyperparameter optimization** 

and also **evolutionary hyperparameter optimization** (not covered here).

One brute force search method often used is **grid search**. In cases where it makes sense to search through a very large space of parameter settings or where you have to do a lot of compute each time you try a setting, it's better to use random search than grid search.

Let's get concrete and try these out on some model trained on some data.

# A data set

We'll look at a data set of housing prices in various districts in California. To make the story clearer, we'll only tune a few hyperparameters of a random forest model.

In [4]:
if colab:
    df = pd.read_csv('https://www.dropbox.com/s/quug4svzdyj5k4j/housing_data.csv?dl=1')

In [6]:
if not colab:
    df = pd.read_csv(DATA/'housing_data.csv')

In [7]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.13,37.67,40.0,1748.0,318.0,914.0,317.0,3.8676,184000.0,NEAR BAY
1,-120.98,37.65,40.0,422.0,63.0,158.0,63.0,7.3841,172200.0,INLAND
2,-118.37,33.87,23.0,1829.0,331.0,891.0,356.0,6.5755,359900.0,<1H OCEAN
3,-117.89,33.90,23.0,1533.0,226.0,693.0,230.0,7.8980,258200.0,<1H OCEAN
4,-122.40,37.76,52.0,1529.0,385.0,1347.0,348.0,2.9312,239100.0,NEAR BAY


Split off some test data:

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('median_house_value', axis=1), 
                                                    df.median_house_value, random_state=42)

In [9]:
X_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
2611,-121.57,39.78,18.0,2221.0,459.0,952.0,440.0,2.0458,INLAND
7883,-120.27,34.72,14.0,1289.0,277.0,693.0,237.0,3.2569,<1H OCEAN
2818,-122.03,37.98,16.0,1209.0,477.0,627.0,482.0,1.3894,NEAR BAY
219,-117.98,33.77,7.0,2252.0,570.0,1576.0,550.0,3.6333,<1H OCEAN
16139,-118.39,33.87,19.0,3303.0,584.0,1329.0,569.0,7.5210,<1H OCEAN


We'll measure performance using root mean squared error:

In [10]:
from sklearn.metrics import mean_squared_error

# Prepare the data

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12384 entries, 2611 to 15795
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           12384 non-null  float64
 1   latitude            12384 non-null  float64
 2   housing_median_age  12384 non-null  float64
 3   total_rooms         12384 non-null  float64
 4   total_bedrooms      12254 non-null  float64
 5   population          12384 non-null  float64
 6   households          12384 non-null  float64
 7   median_income       12384 non-null  float64
 8   ocean_proximity     12384 non-null  object 
dtypes: float64(8), object(1)
memory usage: 967.5+ KB


Some values are missing in `total_bedrooms`. Let's impute them:

In [13]:
from sklearn.impute import SimpleImputer
features = ['total_bedrooms']
imp = SimpleImputer()
X_train.loc[:, features] = imp.fit_transform(X_train[features])
X_test.loc[:, features] = imp.transform(X_test[features])

Featuren `ocean_proximity` er kategorisk, ikke numerisk:

In [14]:
X_train.ocean_proximity.value_counts()

<1H OCEAN     5518
INLAND        3892
NEAR OCEAN    1586
NEAR BAY      1385
ISLAND           3
Name: ocean_proximity, dtype: int64

We have to represent this feature in some good way using **feature encoding**. We can, for example, replace the different categires with the numbers from 0 to 4:

In [15]:
from sklearn.preprocessing import OrdinalEncoder

In [16]:
ord_enc = OrdinalEncoder()

In [17]:
X_train['ocean_proximity'] = ord_enc.fit_transform(X_train[['ocean_proximity']])
X_test['ocean_proximity'] = ord_enc.fit_transform(X_test[['ocean_proximity']])

In [18]:
X_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
2611,-121.57,39.78,18.0,2221.0,459.0,952.0,440.0,2.0458,1.0
7883,-120.27,34.72,14.0,1289.0,277.0,693.0,237.0,3.2569,0.0
2818,-122.03,37.98,16.0,1209.0,477.0,627.0,482.0,1.3894,3.0
219,-117.98,33.77,7.0,2252.0,570.0,1576.0,550.0,3.6333,0.0
16139,-118.39,33.87,19.0,3303.0,584.0,1329.0,569.0,7.5210,0.0


**Feature-skalering:** It's not necessary to scale the features as we plan to use random forest models.

# Our base model

We first create a model using all the standard hyperparameters:

In [19]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

We can check what the standard settings are:

In [21]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

Train:

In [22]:
rf.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

Predict on test:

In [23]:
y_pred = rf.predict(X_test)

Let's check how the model did:

In [24]:
mean_squared_error(y_test, y_pred, squared=False)

50759.940462405924

Now, let's try to improve our baseline model by tweaking hyperparameters:

# Hyperparameter optimization

## Grid search

In grid search we select some paramaters to change, make a set or range of values for each of them, and try every combination. 

We specify our grid as a Python dictionary, or a list of dictionaries if we want to be more specific about parameter combinations to try (for example, "if `n_estimators` is 10, try `max_depth` 2, 3 and 4")

Here's a basic grid:

In [25]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [10, 50, 100, 150, 200],
    'max_depth': [5, 50, None],
    'max_features': [2, 3, None]
}

We can search over all the $5*3*3=45$ different settings, using 3-fold cross validation to check the performance of each one (training the model a total of $45*3 = 135$ times):

In [26]:
gs_reg = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)

In [27]:
gs_reg.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [5, 50, None],
                         'max_features': [2, 3, None],
                         'n_estimators': [10, 50, 100, 150, 200]},
             verbose=1)

We grab the best model

In [28]:
best_reg = gs_reg.best_estimator_

...and check its parameters:

In [29]:
best_reg.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 50,
 'max_features': 3,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [30]:
gs_reg.best_params_

{'max_depth': 50, 'max_features': 3, 'n_estimators': 200}

Let's try it on our test data:

In [31]:
y_pred_best = best_reg.predict(X_test)
mean_squared_error(y_test, y_pred_best, squared=False)

49600.15730378219

An improvement over the baseline.

## Random search

Let's try randomly searching through the space of parameters a specified number of iterations. This can be achieved using `RandomizedSearchCV`.

We can use scikit-learn's `RandomizedSearchCV` in a similar way to `GridSearchCV`: we specify the estimator to use and the parameter grid to search through. But we also specify the number of settings to try, and the method searches randomly through the parameter space that many times.

In [32]:
from sklearn.model_selection import RandomizedSearchCV

In [33]:
#?RandomizedSearchCV

In [34]:
n_iter = 5

In [35]:
rs_reg = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, 
                            n_iter=n_iter, cv=3, verbose=1, n_jobs=-1, random_state=42)

In [36]:
rs_reg.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [5, 50, None],
                                        'max_features': [2, 3, None],
                                        'n_estimators': [10, 50, 100, 150,
                                                         200]},
                   random_state=42, verbose=1)

Since we're only searching through a selection of the settings, this will of course not outperform the brute-force grid search approach.

In [37]:
rs_reg.best_estimator_

RandomForestRegressor(max_features=3, n_estimators=200, n_jobs=-1,
                      random_state=42)

In [38]:
mean_squared_error(y_test,rs_reg.predict(X_test), squared=False)

49600.15730378219

As our data is small it doesn't cost much time to search through *all* the settings in our above parameter grid. That is, using grid search is okay. However, if the parameter space was chosen to be much larger, our data set was more complicated, or our model took much longer to train, random search would be a better approach

> **Your turn!** Try searching over a larger set of parameters. You can f.ex. include `bootstrap`, `min_samples_leaf`, `min_samples_split`, `max_features`. See if you can find a combination that scores higher than what we achieved above. Hint: Take a look at the documentation of `RandomForestRegressor` to learn more about the hyperparameters. 

## More advanced techniques: Bayesian optimization

Instead of searching through parameter settings one at a time without paying attention to what's happened in the past, we can use methods for a more intelligent selection of settings to try next. This can significantly reduce the number of iterations necessary to find good parameter settings and can be useful when looking through a large number of options.

In Bayesian hyperparameter optimization, you start with an objective function that you want to minimize. It could, for example, be mean squared error. You then want to search through a space of possible hyperparameters in an efficient way without having to try all possibilities and without the trials being completely random. 

The basic idea is to somehow build a _model_ of the objective function (more precisely, a probability distribution for the objective function) and then use the model to select the most promising hyperparameter settings. 

Several libraries can achieve this, for example, `hyperopt` https://github.com/hyperopt/hyperopt and `skopt` https://scikit-optimize.github.io.

> Using ML-techniques to automate the construction of machine learning systems is called **AutoML**, a field of research that has gotten significant attention in recent years (there are also multiple products offering AutoML solutions, e.g. Google's [Cloud AutoML](https://cloud.google.com/automl/)).

### `hyperopt`

In [39]:
%pip install hyperopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:000m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.3 MB/s eta 0:00:0031m18.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 8.8 MB/s eta 0:00:00m eta 0:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=cbb574fcf53cf8b984522f0863217f3addb14d794bf33836efed745c5ddc8830
  Stored in directory: /Users/alex/Library/Caches/pip/wheels/22/73/06/557dc4f4ef68179b9d763930d6eec26b88ed7c389b19588a1c
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [40]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

We define a function that we want to minimize during the optimization process. In our case we want to minimize the mean absolute error of our random forest model, as measured using cross validation:

In [41]:
from sklearn.model_selection import cross_val_score

def objective(params):
    # Our objective is to find the hyperparameters that gives the lowest mean squared error 
    # on validation data
    
    print(f"Using parameters {params}")
    # Our model
    model = RandomForestRegressor(**params, n_jobs=-1, random_state=42)
    # The max cross-val score with current paramaters:
    score = cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error").mean()
    # We want to minimize the mean absolute error loss, not its negative:
    loss = -score
    return {'loss': loss, 'status': STATUS_OK}

Then we select a space of parameters over which to search. We set it up similar to the above `param_grid`, using hyperopt to choose random parameter settings:

In [42]:
param_space = {
    'n_estimators': hp.choice('n_estimators', [10, 50, 100, 150, 200]),
    'max_depth': hp.choice('max_depth', [5, 50, None]),
    'max_features': hp.choice('max_features', [2, 3, None])
}

We can manually select a random (stochastic) samples from this space, just to check that it works:

In [43]:
import hyperopt.pyll.stochastic

In [44]:
print(hyperopt.pyll.stochastic.sample(param_space))

{'max_depth': None, 'max_features': 2, 'n_estimators': 100}


Let's minimize our objective. 

In [45]:
# By using the trials object we can check what goes on in each trial:
trials = Trials()

In [46]:
best = fmin(fn=objective, space=param_space, algo=tpe.suggest, 
            max_evals=8, trials=trials)

Using parameters {'max_depth': 5, 'max_features': 3, 'n_estimators': 50}        
Using parameters {'max_depth': 5, 'max_features': None, 'n_estimators': 100}    
Using parameters {'max_depth': None, 'max_features': 3, 'n_estimators': 200}    
Using parameters {'max_depth': None, 'max_features': 2, 'n_estimators': 200}    
Using parameters {'max_depth': 5, 'max_features': None, 'n_estimators': 10}     
Using parameters {'max_depth': None, 'max_features': 3, 'n_estimators': 10}     
Using parameters {'max_depth': None, 'max_features': None, 'n_estimators': 10}  
Using parameters {'max_depth': None, 'max_features': 2, 'n_estimators': 50}     
100%|████████████| 8/8 [00:04<00:00,  1.92trial/s, best loss: 2552335397.110831]


The indices for the best parameters are:

In [47]:
best

{'max_depth': 2, 'max_features': 1, 'n_estimators': 4}

We can use the `space_eval` function to find the corresponding values:

In [48]:
space_eval(param_space, best)

{'max_depth': None, 'max_features': 3, 'n_estimators': 200}

...and train and evaluate a model using those values:

In [49]:
model = RandomForestRegressor(**space_eval(param_space, best), random_state=42, n_jobs=-1)

In [50]:
model.fit(X_train, y_train)
mean_squared_error(y_test,model.predict(X_test), squared=False)

49600.15730378219

> **Your turn!** Make a function that plots the result of each trial. Hint: use the `trials` object. 

> Note: the main component making the above hyperopt approach is the algorithm called `tpe.suggest`. TPE stands for _Tree-structured Parzen Estimator_, and it's what's responsible for building the probability model of our objective function. How it works is, unfortunately, beyond the scope of our course. If you're curious, you can have a look at the [paper introducing the method](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf) or look for popularized explanations online (f.ex. [this one](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f).)

# Your turn!

> **Exercise 1:** Try to search over a much larger set of parameters (and perhaps also over other regression models). Take note of the number of iterations and the total time spent to find a good set of parameters for grid search, random search and hyperopt. 

> **Exercise 2:** Repeat the process for the data sets studied earlier in the course. Feel free to try other models besides random forests. For example the SGDClassifier, or a SVM model.